# Data Science Challenge

In [1]:
import pandas as pd
import numpy as np

# Exercise 1

In [2]:
!ls /home/dsc/Data/challenge/

bookings.csv.bz2  searches.csv.bz2


In [3]:
!bzcat /home/dsc/Data/challenge/bookings.csv.bz2 | head -10000 > bookings_sample.csv


bzcat: I/O or other error, bailing out.  Possible reason follows.
bzcat: Broken pipe
	Input file = /home/dsc/Data/challenge/bookings.csv.bz2, output file = (stdout)


In [4]:
!bzip2 -f bookings_sample.csv

In [5]:
!ls -l

total 780
-rw-rw-r-- 1 dsc dsc 535893 nov 21 00:20 bookings_sample.csv.bz2
-rw-rw-r-- 1 dsc dsc  15880 nov 21 00:20 DS_challenge.ipynb
-rw-rw-r-- 1 dsc dsc 244720 nov 20 19:18 searches_sample.csv.bz2


In [6]:
!bzcat bookings_sample.csv.bz2 | wc -l

10000


In [7]:
import bz2
with bz2.BZ2File('/home/dsc/DS/DS_challenge/bookings_sample.csv.bz2') as file_bz2:
    count = 0
    for line in file_bz2:
        count += 1
print(count)

10000


### Getting number of rows from the whole file (Chunksize)

In [8]:
chunks = pd.read_csv('/home/dsc/Data/challenge/bookings.csv.bz2',
                 compression = 'bz2',
                 sep = '^',
                 chunksize = 100000)

In [9]:
rows = 0
for chunk in chunks:
    rows += len(chunk)
print(rows)

/home/dsc/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (7,8,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


10000010


# Exercise 2

In [4]:
bookings = pd.read_csv('/home/dsc/DS/DS_challenge/bookings_sample.csv.bz2', compression='bz2', sep='^')

In [5]:
bookings.shape

(9999, 38)

In [6]:
bookings.columns

Index(['act_date           ', 'source', 'pos_ctry', 'pos_iata', 'pos_oid  ',
       'rloc          ', 'cre_date           ', 'duration', 'distance',
       'dep_port', 'dep_city', 'dep_ctry', 'arr_port', 'arr_city', 'arr_ctry',
       'lst_port', 'lst_city', 'lst_ctry', 'brd_port', 'brd_city', 'brd_ctry',
       'off_port', 'off_city', 'off_ctry', 'mkt_port', 'mkt_city', 'mkt_ctry',
       'intl', 'route          ', 'carrier', 'bkg_class', 'cab_class',
       'brd_time           ', 'off_time           ', 'pax', 'year', 'month',
       'oid      '],
      dtype='object')

In [7]:
bookings.groupby('arr_port')['pax'].sum().sort_values(ascending=False).head(10)

arr_port
HKG         112
LGA          95
ORD          94
JFK          92
SFO          91
LAX          91
MCO          90
DCA          82
DEN          79
LHR          76
Name: pax, dtype: int64

In [8]:
bookings.loc[:, ['arr_port', 'pax']].isna().sum()

arr_port    0
pax         0
dtype: int64

In [9]:
bookings['arr_port'].value_counts()

LHR         212
LAX         202
JFK         175
ORD         175
SFO         156
CDG         151
LGA         150
MNL         129
HKG         122
PVG         122
MCO         121
IST         118
MIA         109
PHX         108
LAS         107
ATL          98
PEK          94
DEN          89
DCA          88
BOS          88
JED          88
SEA          83
DFW          83
SGN          82
DXB          80
ICN          78
TPA          73
FCO          70
BKK          69
ATH          67
           ... 
LUO           1
KGS           1
BME           1
AOE           1
BEG           1
RAI           1
BTV           1
IVL           1
IXB           1
BDA           1
FWA           1
IGU           1
FSP           1
ARH           1
YXY           1
EWN           1
CUC           1
LAO           1
VSA           1
SPS           1
NAG           1
RAJ           1
IDR           1
ERI           1
YQU           1
QOW           1
HOU           1
HFE           1
JKH           1
RST           1
Name: arr_port, Length: 

Removing white spaces from the columns

In [7]:
bookings.columns = bookings.columns.str.strip()

In [8]:
bookings.columns

Index(['act_date', 'source', 'pos_ctry', 'pos_iata', 'pos_oid', 'rloc',
       'cre_date', 'duration', 'distance', 'dep_port', 'dep_city', 'dep_ctry',
       'arr_port', 'arr_city', 'arr_ctry', 'lst_port', 'lst_city', 'lst_ctry',
       'brd_port', 'brd_city', 'brd_ctry', 'off_port', 'off_city', 'off_ctry',
       'mkt_port', 'mkt_city', 'mkt_ctry', 'intl', 'route', 'carrier',
       'bkg_class', 'cab_class', 'brd_time', 'off_time', 'pax', 'year',
       'month', 'oid'],
      dtype='object')

Unique values for "rloc" column"

In [9]:
len(bookings['rloc'].unique())

3638

### Now let's work with Chunks

In [14]:
chunk_size = 10000
df_chunk = pd.read_csv('/home/dsc/Data/challenge/bookings.csv.bz2',
                     compression = 'bz2',
                     sep = '^',
                     usecols = ['arr_port', 'pax'],
                     chunksize = chunk_size)

In [15]:
def chunk_processing(chunk):
    return chunk.dropna().groupby('arr_port')['pax'].sum()

In [16]:
chunk_list = []

for chunk in df_chunk:
    processed_chunk = chunk_processing(chunk)
    chunk_list.append(processed_chunk)
    
df_concat = pd.concat(chunk_list)

In [17]:
top_10 = df_concat.groupby('arr_port').sum().sort_values(ascending=False).head(10)

In [18]:
top_10

arr_port
LHR         88809.0
MCO         70930.0
LAX         70530.0
LAS         69630.0
JFK         66270.0
CDG         64490.0
BKK         59460.0
MIA         58150.0
SFO         58000.0
DXB         55590.0
Name: pax, dtype: float64

In [23]:
top_10 = top_10.reset_index()

In [24]:
top_10

,arr_port,pax
0,LHR,88809.0
1,MCO,70930.0
2,LAX,70530.0
3,LAS,69630.0
4,JFK,66270.0
5,CDG,64490.0
6,BKK,59460.0
7,MIA,58150.0
8,SFO,58000.0
9,DXB,55590.0


And... Eventually I got it!! <br>
Now let's save it into a .csv and commit the new changes

In [27]:
top_10.to_csv('002_out.csv')

# Exercise 3

In [ ]:
!bzcat /home/dsc/Data/challenge/searches.csv.bz2 | head -10000 > searches_sample.csv

In [ ]:
!bzip2 -f searches_sample.csv

In [ ]:
!ls -l

In [ ]:
searches = pd.read_csv('searches_sample.csv.bz2', compression='bz2', sep='^')

In [ ]:
searches.shape

In [ ]:
searches['Date'] = pd.to_datetime(searches['Date'], format='%Y-%m-%d')

In [ ]:
searches = pd.read_csv('/home/dsc/Data/')

In [ ]:
chunks_s = pd.read_csv('/home/dsc/Data/challenge/searches.csv.bz2',
                     compression = 'bz2',
                     sep = '^',
                     chunksize = 100000)